In [1]:
!pip uninstall -y unsloth transformers accelerate peft bitsandbytes
!pip install -U pip
!pip install -q "unsloth[colab-new]"


Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.9.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.


In [1]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="train.jsonl",
    split="train"
)


In [2]:
def format_example(example):
    return {
        "text": f"""<s>[INST] {example['instruction']}

{example['input']} [/INST]
{example['output']}</s>"""
    }
dataset = dataset.map(format_example)
dataset = dataset.remove_columns(
    [col for col in dataset.column_names if col != "text"]
)


In [4]:
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    max_seq_length=2048,
    dtype=torch.float16,      # IMPORTANT for T4
    load_in_4bit=True,        # QLoRA
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [7]:
def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens
dataset = dataset.map(
    tokenize_function,
    batched=False,
    remove_columns=["text"],
)
print(dataset.column_names)


Map:   0%|          | 0/318 [00:00<?, ? examples/s]

['input_ids', 'attention_mask', 'labels']


In [5]:
from peft import LoraConfig, get_peft_model
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [9]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./unsloth-mistral-action",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,  # 🔑 FIX
)



In [10]:
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    tokenizer=tokenizer,
)


/tmp/ipython-input-1382102699.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 318 | Num Epochs = 3 | Total steps = 120
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 3,407,872 of 7,245,139,968 (0.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.106300
20,0.896100
30,0.752000
40,0.722300
50,0.621000
60,0.600100
70,0.554700
80,0.536500
90,0.505800
100,0.520300


TrainOutput(global_step=120, training_loss=0.6547240495681763, metrics={'train_runtime': 1909.1562, 'train_samples_per_second': 0.5, 'train_steps_per_second': 0.063, 'total_flos': 2.4351369285648384e+16, 'train_loss': 0.6547240495681763, 'epoch': 3.0})

In [12]:
model.save_pretrained("meeting-action-lora")
tokenizer.save_pretrained("meeting-action-lora")


('meeting-action-lora/tokenizer_config.json',
 'meeting-action-lora/special_tokens_map.json',
 'meeting-action-lora/chat_template.jinja',
 'meeting-action-lora/tokenizer.model',
 'meeting-action-lora/added_tokens.json',
 'meeting-action-lora/tokenizer.json')

In [8]:
from unsloth import FastLanguageModel
import torch

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model.eval()


==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layer

In [9]:
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    "meeting-action-lora",  # your saved LoRA directory
)

model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [10]:
FastLanguageModel.for_inference(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [14]:
prompt = """<s>[INST]
You are a system that extracts action items from meeting transcripts.

Rules:
- Output ONLY valid JSON
- Do NOT add explanations, headings, or bullet points
- Do NOT include text outside JSON
- Use this schema exactly:

{
  "action_items": [
    {
      "action": string,
      "owner": string | null,
      "deadline": string | null
    }
  ]
}

Meeting transcript:
Alex: Please follow up with the client by Friday.
Sam: I will send the email tomorrow.
[/INST]"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **tokenizer(prompt, return_tensors="pt").to("cuda"),
    max_new_tokens=512,
    temperature=0.0,   # 🔑 critical
    do_sample=False,  # 🔑 critical
)



In [15]:
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Remove everything before the model answer
answer = decoded.split('[/INST]')[-1].strip()

print(answer)


{
  "action_items": [
    {
      "action": "Follow up with the client",
      "owner": null,
      "deadline": "Friday"
    },
    {
      "action": "Send the email",
      "owner": "Sam",
      "deadline": "tomorrow"
    }
  ]
}


Schema Validity using Pydantic

In [11]:
eval_ds = load_dataset(
    "json",
    data_files="eval.jsonl",
    split="train"
)

In [12]:
from pydantic import BaseModel, ValidationError, ConfigDict
from typing import List, Optional

class ActionItem(BaseModel):
    action: str
    owner: Optional[str]
    deadline: Optional[str]

    model_config = ConfigDict(extra="ignore")

class ActionItemsOutput(BaseModel):
    action_items: List[ActionItem]

    model_config = ConfigDict(extra="ignore")


In [27]:
import json
import re

import json
import ast

def extract_json(text: str):
    if not isinstance(text, str):
        return None

    text = text.strip()

    # 1️⃣ Remove everything before [/INST]
    if "[/INST]" in text:
        text = text.split("[/INST]", 1)[1].strip()

    # 2️⃣ Try JSON
    try:
        return json.loads(text)
    except Exception:
        pass

    # 3️⃣ Try Python literal (THIS is your case)
    try:
        return ast.literal_eval(text)
    except Exception:
        return None


In [14]:
def clean_action_items(data: dict) -> dict:
    if "action_items" not in data:
        return data

    cleaned = []
    for item in data["action_items"]:
        if isinstance(item, dict):
            item = dict(item)
            item.pop("action_items", None)  # 🔑 REMOVE LEAK
            cleaned.append(item)

    data["action_items"] = cleaned
    return data


In [21]:
import json
def is_schema_valid(output):
    try:
        print("TYPE OF OUTPUT:", type(output))

        if isinstance(output, dict):
            print("→ Branch: dict")
            data = clean_action_items(output)
            print("CLEANED DATA:", data)
            ActionItemsOutput(**data)
            return True

        if isinstance(output, str):
            print("→ Branch: str")
            data = extract_json(output)
            print("EXTRACTED:", data)
            if data is None:
                return False
            data = clean_action_items(data)
            print("CLEANED DATA:", data)
            ActionItemsOutput(**data)
            return True

        print("→ Branch: neither")
        return False

    except ValidationError as e:
        print(" VALIDATION ERROR:", e)
        return False


In [25]:
import torch

def run_inference(model, tokenizer, instruction, transcript):
    prompt = f"""<s>[INST] {instruction}

{transcript} [/INST]"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.0,
        do_sample=False,
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    data = extract_json(decoded)

    if data is None:
        print("⚠️ extract_json FAILED")
        return None

    return data


In [17]:
def schema_validity_rate(model, tokenizer, eval_ds):
    valid = 0

    for ex in eval_ds:
        output = run_inference(
            model,
            tokenizer,
            ex["instruction"],
            ex["input"],
        )

        if is_schema_valid(output):
            valid += 1

    return valid / len(eval_ds)


In [28]:
def debug_schema_examples(model, tokenizer, eval_ds, n=1):
    for i in range(min(n, len(eval_ds))):
        ex = eval_ds[i]

        print("=" * 80)
        print(f"Example {i+1}")

        print("\n📄 TRANSCRIPT:")
        print(ex["input"][:500], "...\n")

        output = run_inference(
            model,
            tokenizer,
            ex["instruction"],
            ex["input"],
        )

        print("🤖 RAW MODEL OUTPUT:")
        print(output)

        valid = is_schema_valid(output)
        print("\n✅ SCHEMA VALID:", valid)

        # Optional: show parsed JSON if valid
        if valid:
            if isinstance(output, dict):
                parsed = output
            else:
                parsed = extract_json(output)

            print("\n🧩 PARSED JSON:")
            print(parsed)

        print("\n")
debug_schema_examples(model, tokenizer, eval_ds, n=1)


Example 1

📄 TRANSCRIPT:
**Meeting Transcript:**

**Participants:**
- Sarah (Project Manager)
- Mark (Marketing Lead)
- Lisa (Product Designer)
- Tom (Sales Director)

---

**Sarah:** Good morning, everyone. Let's kick off by reviewing our current project status. Mark, can you start with the marketing updates?

**Mark:** Sure, Sarah. We’ve finalized the campaign strategy and are ready to launch next week. I'll send out the detailed plan by tomorrow.

**Sarah:** Great, thanks, Mark. Lisa, how's the product design coming  ...


--- RAW DECODED TEXT ---
[INST] Extract action items from the meeting transcript.

**Meeting Transcript:**

**Participants:**
- Sarah (Project Manager)
- Mark (Marketing Lead)
- Lisa (Product Designer)
- Tom (Sales Director)

---

**Sarah:** Good morning, everyone. Let's kick off by reviewing our current project status. Mark, can you start with the marketing updates?

**Mark:** Sure, Sarah. We’ve finalized the campaign strategy and are ready to launch next week. I

In [29]:
rate = schema_validity_rate(model, tokenizer, eval_ds)
print(f"Schema validity: {rate:.2%}")



--- RAW DECODED TEXT ---
[INST] Extract action items from the meeting transcript.

**Meeting Transcript:**

**Participants:**
- Sarah (Project Manager)
- Mark (Marketing Lead)
- Lisa (Product Designer)
- Tom (Sales Director)

---

**Sarah:** Good morning, everyone. Let's kick off by reviewing our current project status. Mark, can you start with the marketing updates?

**Mark:** Sure, Sarah. We’ve finalized the campaign strategy and are ready to launch next week. I'll send out the detailed plan by tomorrow.

**Sarah:** Great, thanks, Mark. Lisa, how's the product design coming along?

**Lisa:** The prototype is ready for testing. I’ll coordinate with the engineering team to start user testing by Friday.

**Tom:** That sounds good, Lisa. Once the testing is done, I can prepare the sales materials. Just let me know when you have the results.

**Lisa:** Will do, Tom. I’ll keep you updated on the progress.

**Sarah:** Excellent. Now, about the upcoming trade show. We need to finalize our b

Schema validity: 97.14%

Prompt-only baseline (very important)

In [30]:
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)


==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [31]:
rate = schema_validity_rate(base_model, base_tokenizer, eval_ds)
print(f"Schema validity: {rate:.2%}")



--- RAW DECODED TEXT ---
[INST] Extract action items from the meeting transcript.

**Meeting Transcript:**

**Participants:**
- Sarah (Project Manager)
- Mark (Marketing Lead)
- Lisa (Product Designer)
- Tom (Sales Director)

---

**Sarah:** Good morning, everyone. Let's kick off by reviewing our current project status. Mark, can you start with the marketing updates?

**Mark:** Sure, Sarah. We’ve finalized the campaign strategy and are ready to launch next week. I'll send out the detailed plan by tomorrow.

**Sarah:** Great, thanks, Mark. Lisa, how's the product design coming along?

**Lisa:** The prototype is ready for testing. I’ll coordinate with the engineering team to start user testing by Friday.

**Tom:** That sounds good, Lisa. Once the testing is done, I can prepare the sales materials. Just let me know when you have the results.

**Lisa:** Will do, Tom. I’ll keep you updated on the progress.

**Sarah:** Excellent. Now, about the upcoming trade show. We need to finalize our b

Schema validity: 0.00%